In [7]:
#imports
import torchvision.transforms as transforms
#from torchvision.models import resnet50, ResNet50_Weights
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import numpy as np
import random
import time
import copy
import os
import csv

In [8]:
#Define functions: set device (GPU | CPU) and Seed
def set_device():
  '''Set device to use GPU if possible, else use CPU'''
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("WARNING: For this notebook to perform best, "
        "if possible, in the menu under `Runtime` -> "
        "`Change runtime type.`  select `GPU` ")
  else:
    print("GPU is enabled in this notebook.")

  return device

def set_seed(seed=None, seed_torch=True):
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  if seed_torch:
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

  print(f'Random seed {seed} has been set.')

def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

#Set Device and Seed
device = set_device()
set_seed(seed=2022)

Random seed 2022 has been set.


In [9]:
#get directories
val_dir = './experiment1_valid_nma'
train_dir = './experiment1_train_nma'
test_dir = './experiment1_test_nma'
result_dir = '../results'
if not os.path.exists(result_dir):
  os.makedirs(result_dir)

checkpoint_dir = './checkpoint/'
if not os.path.exists(checkpoint_dir):
  os.makedirs(checkpoint_dir)

#Set Hyperparameters
batch_size = 128
base_learning_rate = 1e-3
best_accuracy = 0
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
max_epoch = 10
use_cuda = torch.cuda.is_available()
freeze_layers = True # freeze all but the output layer (fc)
classes = np.unique(np.array(os.listdir(train_dir) + os.listdir(test_dir)))
classes = classes[classes != '.DS_Store']
num_classes = len(classes)

In [10]:
# Transform Properties
data_transforms = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

# Load Data
train_dataset = datasets.ImageFolder(train_dir,data_transforms)
val_dataset = datasets.ImageFolder(val_dir, data_transforms)
test_dataset = datasets.ImageFolder(test_dir,data_transforms)

# Data Loaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True) # DONT NEED TO BATCH THE TEST SET

# Check Train and Test Data Properties
train_images, train_labels = next(iter(train_dataloader))
val_images, val_labels = next(iter(val_dataloader)) 
test_images, test_labels = next(iter(test_dataloader)) 

In [13]:
# Use pytorch models using pre-trained weights
net = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
net.fc = nn.Linear(net.fc.in_features,num_classes)

In [14]:
# Define Loss and Optimization Functions
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=base_learning_rate, momentum=0.9, weight_decay=1e-4)


In [15]:
# Define training and test functions
def train(net, epoch, use_cuda=True):
  '''Train the model on the test set'''
  print('\nEpoch: %d' % epoch)
  net.train()
  train_loss = 0
  correct = 0
  total = 0
  for batch_idx, (inputs, targets) in enumerate(train_dataloader):
    if use_cuda:
      inputs, targets = inputs.cuda(), targets.cuda()

    optimizer.zero_grad()
    inputs, targets = Variable(inputs), Variable(targets)
    outputs = net(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    _, predicted = torch.max(outputs.data, 1)
    total += targets.size(0)
    correct += predicted.eq(targets.data).cpu().sum()

    if batch_idx % 500 == 0:
      print(batch_idx, len(train_dataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
          % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
  return (train_loss/batch_idx, 100.*correct/total)


def test(net, epoch, outModelName, use_cuda=True):
  '''Test the model on the validation set'''
  global best_acc
  net.eval()
  test_loss, correct, total = 0, 0, 0
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(val_dataloader):
      if use_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()

      outputs = net(inputs)
      loss = criterion(outputs, targets)

      test_loss += loss.item()
      _, predicted = torch.max(outputs.data, 1)
      total += targets.size(0)
      correct += predicted.eq(targets.data).cpu().sum()

      if batch_idx % 200 == 0:
        print(batch_idx, len(val_dataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

  # Save checkpoint
  accuracy = 100.*correct/total
  if accuracy > best_accuracy:
    best_accurary = accuracy
    checkpoint(net, accuracy, epoch, outModelName)
  return (test_loss/batch_idx, 100.*correct/total)

#Define checkpoint and learning rate adjustment
def checkpoint(model, acc, epoch, outModelName):
  '''Save check point'''
  print('Saving..')
  state = {
      'state_dict': model.state_dict(),
      'acc': acc,
      'epoch': epoch,
      'rng_state': torch.get_rng_state()
  }
  if not os.path.isdir('checkpoint'):
      os.mkdir('checkpoint')
  torch.save(state, f'./checkpoint/{outModelName}.t7')

def adjust_learning_rate(optimizer, epoch):
  '''Decrease the learning rate at 100 and 150 epoch'''
  lr = base_learning_rate
  if epoch <= 9 and lr > 0.1:
    # warm-up training for large minibatch
    lr = 0.1 + (base_learning_rate - 0.1) * epoch / 10.
  if epoch >= 100:
    lr /= 10
  if epoch >= 150:
    lr /= 10
  for param_group in optimizer.param_groups:
    param_group['lr'] = lr

In [16]:
for name, param in net.named_parameters():
  if freeze_layers:
    if 'fc' not in name:
      param.requires_grad=False
  else:
    param.requires_grad=True

In [17]:
outModelName = 'pretrain'
logName = result_dir + net.__class__.__name__ + '_' + outModelName + '.csv'

if not os.path.exists(logName):
  with open(logName, 'w') as logfile:
      logwriter = csv.writer(logfile, delimiter=',')
      logwriter.writerow(['epoch', 'train loss', 'train acc', 'test loss', 'test acc'])

for epoch in range(start_epoch, max_epoch):
  adjust_learning_rate(optimizer, epoch)
  train_loss, train_acc = train(net, epoch)#, use_cuda=use_cuda)

  test_loss, test_acc = test(net, epoch, outModelName)#, use_cuda=use_cuda)
  
  with open(logName, 'a') as logfile:
    logwriter = csv.writer(logfile, delimiter=',')
    logwriter.writerow([epoch, train_loss, train_acc.item(), test_loss, test_acc.item()])
  print(f'Epoch: {epoch} | train acc: {train_acc} | test acc: {test_acc}')


Epoch: 0


AssertionError: Torch not compiled with CUDA enabled